In [1]:
import torch
import numpy as np
import pandas as pd
from sentence_transformers import CrossEncoder, SentenceTransformer, util
from underthesea import sent_tokenize
from vinorm import TTSnorm
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import io
import soundfile as sf  # Thư viện xử lý âm thanh
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from pydub import AudioSegment
from io import BytesIO
import librosa
from Xu_ly_text import Xu_ly_text, Xu_ly_text_de_doc
import re
from dotenv import load_dotenv
import os
# from huggingface_hub import login

load_dotenv()

device = "cuda" if torch.cuda.is_available() else "cpu"

# # Get the Hugging Face access token from environment variables
# hf_token = os.getenv("PROJECTCB1_HUGGINGFACE_ACCESS_TOKEN")

# # Log in to Hugging Face using the access token
# if hf_token:
#     login(token=hf_token)
# else:
#     print(
#         "Access token not found. Please set the HUGGINGFACE_ACCESS_TOKEN in your .env file."
#     )

eb_model_path = os.getenv("PROJECTCB1_EMBEDDING_MODEL")
embeddings_path = os.getenv("PROJECTCB1_EMBEDDING_DATA_PATH")


# Hàm nội bộ
def load_embedding_model(embedding_model_path):
    embedding_model = SentenceTransformer(
        model_name_or_path=embedding_model_path, device=device
    )
    return embedding_model


def load_reranking_model(pr_model_path):
    pr_model = CrossEncoder(model_name=pr_model_path, device=device, trust_remote_code=True)
    return pr_model


def load_embeddings(embeddings_path):
    text_chunks_and_embedding_df = pd.read_csv(embeddings_path)
    text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df[
        "embedding"
    ].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
    pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")
    embeddings = torch.tensor(
        np.array(text_chunks_and_embedding_df["embedding"].tolist()),
        dtype=torch.float32,
    ).to(device)
    return embeddings, pages_and_chunks


def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def load_model_tts(xtts_checkpoint, xtts_config, xtts_vocab):
    clear_gpu_cache()
    config = XttsConfig()
    config.load_json(xtts_config)
    XTTS_MODEL = Xtts.init_from_config(config)

    use_deepspeed = torch.cuda.is_available()

    XTTS_MODEL.load_checkpoint(
        config,
        checkpoint_path=xtts_checkpoint,
        vocab_path=xtts_vocab,
        use_deepspeed=use_deepspeed,
    )
    if torch.cuda.is_available():
        XTTS_MODEL.cuda()
    return XTTS_MODEL


def normalize_vietnamese_text(text):
    text = Xu_ly_text_de_doc(text)
    text = (
        TTSnorm(text, unknown=False, lower=False, rule=True)
        .replace("  ", " ")
        .replace(":", ".")
        .replace("!.", "!")
        .replace("?.", "?")
        .replace(" .", ".")
        .replace(" ,", ",")
        .replace('"', "")
        .replace("'", "")
        .replace("+", " ")
        .replace("..", ".")
        .replace("AI", "Ây Ai")
        .replace("A.I", "Ây Ai")
    )
    return text.lower()


def split_sentences(text, max_length=245):
    text = (
        text.replace("\n", ". ").replace(";", ".").replace("?", ".").replace("!", ".")
    )

    sentences = re.findall(r"[^,.]+[,.]", text)
    grouped_sentences = []
    current_group = ""

    for sentence in sentences:
        # Nếu thêm câu vào mà không vượt quá giới hạn max_length
        if len(current_group) + len(sentence) + 1 < max_length:
            if current_group:
                current_group += " " + sentence  # Ghép câu mới vào câu trước đó
            else:
                current_group = sentence  # Câu đầu tiên của nhóm
        elif len(sentence) > max_length:  # Xử lý
            if current_group:
                grouped_sentences.append(current_group)
                current_group = ""
            k = 0
            tamthoi = []
            for i in sentence.split(" "):
                tamthoi += [i]
                if len(tamthoi) >= 40:
                    grouped_sentences += [" ".join(tamthoi)]
                    tamthoi = []
            if tamthoi:
                grouped_sentences += [" ".join(tamthoi)]
        else:
            grouped_sentences.append(current_group)  # Thêm nhóm vào list
            current_group = sentence  # Khởi tạo nhóm mới với câu hiện tại

    if current_group:
        grouped_sentences.append(current_group)  # Thêm nhóm cuối cùng vào list

    return grouped_sentences


# Khai báo các mô hình

print("Loading models... ")
# Load model embedding

embedding_model = load_embedding_model(eb_model_path)

# Load reranking
# rr_model_path = "itdainb/PhoRanker"
# reranking_model = load_reranking_model(rr_model_path)

# Dowload TTS capleaf/viXTTS
# from huggingface_hub import snapshot_download

# snapshot_download(
#     repo_id="capleaf/viXTTS", repo_type="model", local_dir="Model/TTS_model"
# )

tts_model_path = os.getenv("PROJECTCB1_TTS_MODEL")
# Load model TTS capleaf/viXTTS
vixtts_model = load_model_tts(
    xtts_checkpoint=f"{tts_model_path}/model.pth",
    xtts_config=f"{tts_model_path}/config.json",
    xtts_vocab=f"{tts_model_path}/vocab.json",
)

embeddings, pages_and_chunks = load_embeddings(embeddings_path)  # Load embeddings
reference_audio = os.getenv("PROJECTCB1_REFERENCE_AUDIO")  # Mẫu giọng nói

# Load model STT nguyenvulebinh/wav2vec2-base-vietnamese-250h

stt_model_path = os.getenv("PROJECTCB1_STT_MODEL")
processor = Wav2Vec2Processor.from_pretrained(stt_model_path)
model = Wav2Vec2ForCTC.from_pretrained(stt_model_path).to(device)
print("Models Loaded!")

# processor.save_pretrained(stt_model_path)
# model.save_pretrained(stt_model_path)
# Hàm sử dụng cho API


def run_stt(audio_bytes):
    # Đọc tệp âm thanh từ byte
    audio = AudioSegment.from_file(BytesIO(audio_bytes))

    # Chuyển đổi âm thanh thành mảng numpy
    samples = np.array(audio.get_array_of_samples())

    # Đảm bảo là mono (1 kênh)
    if audio.channels > 1:
        samples = samples.reshape((-1, audio.channels))
        samples = samples.mean(
            axis=1
        )  # Lấy trung bình giá trị của tất cả các kênh để chuyển sang mono

    # Chuẩn hóa lại tần số mẫu về 16000 Hz
    samples_16k = librosa.resample(
        samples.astype(np.float32), orig_sr=audio.frame_rate, target_sr=16000
    )

    # Tokenize dữ liệu đầu vào
    input_values = processor(
        samples_16k, return_tensors="pt", padding="longest", sampling_rate=16000
    ).input_values

    # Chuyển sang GPU và chuyển đổi sang float
    input_values = input_values.to(device).float()

    # Lấy kết quả dự đoán từ mô hình
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # Giải mã kết quả dự đoán thành văn bản
    transcription = processor.batch_decode(predicted_ids)[0]
    text = Xu_ly_text(transcription)
    return text


def run_tts(text, lang="vi"):
    if vixtts_model is None or not reference_audio:
        return "You need to run the previous step to load the model !!", None, None

    gpt_cond_latent, speaker_embedding = vixtts_model.get_conditioning_latents(
        audio_path=reference_audio,
        gpt_cond_len=vixtts_model.config.gpt_cond_len,
        max_ref_length=vixtts_model.config.max_ref_len,
        sound_norm_refs=vixtts_model.config.sound_norm_refs,
    )

    # Chuẩn hóa
    tts_text = normalize_vietnamese_text(text)
    tts_texts = split_sentences(tts_text)
    print(tts_texts)
    wav_chunks = []
    for text in tts_texts:
        if text.strip() == "":
            continue

        wav_chunk = vixtts_model.inference(
            text=text,
            language=lang,
            gpt_cond_latent=gpt_cond_latent,
            speaker_embedding=speaker_embedding,
            temperature=0.01,  # 0.3
            length_penalty=1.0,  # 1.0
            repetition_penalty=50.0,  # 10.0
            top_k=5,  # 30
            top_p=0.95,  # 0.85
        )

        keep_len = -1
        wav_chunk["wav"] = torch.tensor(wav_chunk["wav"][:keep_len])
        wav_chunks.append(wav_chunk["wav"])

    out_wav = (
        torch.cat(wav_chunks, dim=0).squeeze(0).cpu().numpy()
    )  # Chuyển sang numpy array

    # Chuyển đổi Tensor thành định dạng WAV
    buffer = io.BytesIO()

    # Ghi âm thanh vào buffer, đảm bảo dữ liệu đầu vào là numpy array và định dạng đúng
    try:
        sf.write(buffer, out_wav, 24000, format="WAV")
        buffer.seek(0)
        wav_data = buffer.read()
    except Exception as e:
        print(f"Error writing WAV file: {e}")
        return None, None, None

    return wav_data




/home/andv/important/chatbot_rag/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Loading models... 


/home/andv/important/chatbot_rag/.venv/src/tts/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)
GP

[2024-10-25 15:10:53,994] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2024-10-25 15:10:54,582] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.15.2, git-hash=unknown, git-branch=unknown
[2024-10-25 15:10:54,583] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter replace_method is deprecated. This parameter is no longer needed, please remove from your call to DeepSpeed-inference
[2024-10-25 15:10:54,583] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2024-10-25 15:10:54,584] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1
[2024-10-25 15:10:54,681] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layers': -1, 'dtype': torch.float32, 'pre_layer_norm': True, 'norm_type': <NormType.LayerNorm: 1>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_siz

Using /home/andv/.cache/torch_extensions/py310_cu124 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/andv/.cache/torch_extensions/py310_cu124/transformer_inference/build.ninja...
/home/andv/important/chatbot_rag/.venv/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
Time to load transformer_inference op: 0.04592084884643555 seconds


Loading extension module transformer_inference...


Models Loaded!


In [2]:
def retrieve_relevant_resources(query: str, n_resources_to_return: int = 5, threshold: int =0.01):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    # Use cosine similarity instead of dot score
    cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    
    # Get all scores and corresponding indices, then filter based on score > 0.5
    scores, indices = torch.topk(input=cosine_scores, k=n_resources_to_return)
    filtered_scores_indices = [(score.item(), index.item()) for score, index in zip(scores, indices) if score.item() > threshold]
    
    # Extract the scores and indices after filtering
    filtered_indices = [index for _, index in filtered_scores_indices]
    
    # Take top 'n_resources_to_return' from the filtered list
    # top_scores = filtered_scores[:n_resources_to_return]
    top_indices = filtered_indices[:n_resources_to_return]
    
    context_items = [pages_and_chunks[i] for i in top_indices]
    results = [item["Final_Answer"] for item in context_items]
    # pr_results = reranking_model.rank(query, results, return_documents=True, top_k=3)
    
    return results

# Example usage:
# results, scores = retrieve_relevant_resources("What is the capital of Japan?", n_resources_to_return=3)
# print(results, scores)


In [24]:
# # Function to format the prompt
# def prompt_formatter(query: str, results: list) -> str:
#     context = "* " + "\n\n* ".join(results)
#     base_prompt = """Hãy cho bản thân không gian để suy nghĩ bằng cách trích xuất các đoạn văn có liên quan từ ngữ cảnh dưới đây trước khi trả lời câu hỏi:
# \n{context}
# \nCác đoạn văn có liên quan: <trích xuất các đoạn văn có liên quan từ ngữ cảnh tại đây>
# Câu hỏi: {query}
# """
#     formatted_prompt = base_prompt.format(context=context, query=query)
#     prompt = f"### Câu hỏi: {formatted_prompt}\n\n### Trả lời:" 
#     return prompt



In [24]:
# def prompt_formatter(query: str, results: list) -> str:
#     context = "* " + "\n\n* ".join(results)
#     base_prompt = """Hãy để cho bản thân để suy nghĩ bằng cách sử dụng các mục ngữ cảnh sau để trả lời câu hỏi của người dùng:\n\n{context}\n\nCâu hỏi của người dùng: {query}"""
#     formatted_prompt = base_prompt.format(context=context, query=query)
#     prompt = f"""<|im_start|>system
# Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
# <|im_end|>
# <|im_start|>user
# {formatted_prompt}<|im_end|>
# <|im_start|>assistant
# """
#     return prompt

In [ ]:
# Function to format the prompt
def prompt_formatter(query: str, results: list) -> str:
    context = "* " + "\n\n* ".join(results)
    base_prompt = """Hãy cho bản thân không gian để suy nghĩ bằng cách trích xuất các đoạn văn có liên quan từ ngữ cảnh dưới đây trước khi trả lời câu hỏi:
\n{context}
\nCác đoạn văn có liên quan: <trích xuất các đoạn văn có liên quan từ ngữ cảnh tại đây>
Câu hỏi: {query}
"""
    prompt = base_prompt.format(context=context, query=query)
    return prompt

In [25]:
query = "Nội dung dự án 1 là gì?"
results = retrieve_relevant_resources(query)
prompt = prompt_formatter(query, results)

In [26]:
print(prompt)

### Câu hỏi: Hãy cho bản thân không gian để suy nghĩ bằng cách trích xuất các đoạn văn có liên quan từ ngữ cảnh dưới đây trước khi trả lời câu hỏi:

* Nội dung cụ thể của Dự án 1 là gì? "Nội dung cụ thể của Dự án 1 bao gồm:
- Nội dung số 01: Hỗ trợ đất ở: Căn cứ quỹ đất, hạn mức đất ở và khả năng ngân sách, Ủy ban nhân dân cấp tỉnh xem xét, quyết định giao đất để làm nhà ở cho các đối tượng nêu trên phù hợp với điều kiện, tập quán ở địa phương và pháp luật về đất đai, cụ thể:
+ Ở những nơi có điều kiện về đất đai, chính quyền địa phương sử dụng số tiền hỗ trợ từ ngân sách để tạo mặt bằng, làm hạ tầng kỹ thuật để cấp đất ở cho các đối tượng được thụ hưởng;
+ Ở các địa phương không có điều kiện về đất đai, chính quyền địa phương bố trí kinh phí hỗ trợ cho người dân tự ổn định chỗ ở theo hình thức xen ghép.
- Nội dung số 02: Hỗ trợ nhà ở: Hỗ trợ xây dựng 01 căn nhà theo phong tục tập quán của địa phương, định mức tính theo xây dựng 01 căn nhà cấp 4 đảm bảo 3 cứng (nền cứng, khung - tường 

In [10]:
# coding: utf8
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

model_path = "vinai/PhoGPT-4B-Chat"  
# model_path = "vilm/vinallama-2.7b-chat"
# model_path = "/home/andv/important/Chatbot/finetune_llm/phogpt-mergerd-with-config-1"

# model_path = "/home/andv/important/Chatbot/finetune"

# config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)  
# config.init_device = "cuda"
# config.attn_config['attn_impl'] = 'flash' # If installed: this will use either Flash Attention V1 or V2 depending on what is installed

model = AutoModelForCausalLM.from_pretrained(model_path,
                                            #  config=config, 
                                             torch_dtype=torch.bfloat16, 
                                             trust_remote_code=True).to("cuda")

# model = AutoModelForCausalLM.from_pretrained(model_path, config = config, trust_remote_code=True).to("cuda")
# If your GPU does not support bfloat16:
# model = AutoModelForCausalLM.from_pretrained(model_path, config=config, torch_dtype=torch.float16, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)  


/home/andv/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/home/andv/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))


In [27]:
def ask(query:str):
    results = retrieve_relevant_resources(query=query, n_resources_to_return=3, threshold=0.5)
    input_prompt = prompt_formatter(query=query, results=results)
    input_ids = tokenizer(input_prompt, return_tensors="pt")  

    outputs = model.generate(  
        inputs=input_ids["input_ids"].to("cuda"),  
        attention_mask=input_ids["attention_mask"].to("cuda"),  
        do_sample=True,  
        temperature=0.8,  
        top_k=50,  
        top_p=0.9,  
        max_new_tokens=1024,  
        eos_token_id=tokenizer.eos_token_id,  
        pad_token_id=tokenizer.pad_token_id  
    )  

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
    # response = tokenizer.decode(outputs[0])
    # response = response.split("### Trả lời:")[1]
    return response

In [ ]:
print(ask("Nội dung dự án 1 là gì, liệt kê các nội dung đấy?"))